<a href="https://colab.research.google.com/github/alrb2005/MechaineLearing-0716-/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# from google.colab import drive
# drive.mount('/content/drive')

In [86]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/AI課程/機器學習/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/AI課程/機器學習/chinese_news_test.zip")
f.extractall("test")

In [87]:
import glob
import os 
import pandas as pd

def getdf(targetdn):
  datas = {
        "path":[],
        "label":[]

  }

  base = ""
  p = os.path.join(base, targetdn, "*")
  for dn in glob.glob(p):
    name = os.path.split(dn)[-1]
    os.path.split(dn)[-1]
    fn = os.path.join(dn, "*.[tT][xX][tT]")
    fns = glob.glob(fn)
    datas["label"] = datas["label"] + [name] * len(fns)
    datas["path"] = datas["path"] + fns
  return pd.DataFrame(datas)

In [88]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")
train_df

,path,label
0,train/chinese_news_trans/軍事/813.TXT,軍事
1,train/chinese_news_trans/軍事/8223.txt,軍事
2,train/chinese_news_trans/軍事/8123.TXT,軍事
3,train/chinese_news_trans/軍事/871.TXT,軍事
4,train/chinese_news_trans/軍事/889.TXT,軍事
...,...,...
2632,train/chinese_news_trans/政治/954.TXT,政治
2633,train/chinese_news_trans/政治/9463.TXT,政治
2634,train/chinese_news_trans/政治/9195.TXT,政治
2635,train/chinese_news_trans/政治/9444.TXT,政治


In [89]:
def read_content(fn):
  with open(fn, "r", encoding="utf-8") as f:
    content = f.read()
  return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)


訓練模型

In [90]:
u = train_df["label"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trans)
y_test = test_df["label"].replace(trans)

In [91]:
x_train = train_df["content"]
x_test = test_df["content"]


In [92]:
import jieba
p = train_df.iloc[0]["content"]
" ".join(jieba.cut(p))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.010 seconds.
Prefix dict has been built successfully.


'                      古巴 宣 佈 舉行 軍事 演習 \n 新華社 哈瓦那 ５ 月 １ 日電 古巴 革命 武裝部 今晚 發表公 \n                           報宣 佈 ， 古巴 將從 ５ 月 ２ 日起 舉行 軍事 演習 ， 以 對 抗美國 \n                           舉行 的 針對 古巴 的 大規模 軍事 演習 。 \n                                   公報 說 ， 參加 這次 代號 為 “ 古巴 之盾 ” 軍事 演習 的 ， \n                           除了 古巴 正規部隊 之外 ， 還有 民兵 。 \n                                   據 古巴 革命 武裝部 ４ 月 ２ ９ 日 揭露 ， 美國 從 ５ 月 １ 日 \n                           至 ５ 日 在 鄰近 古巴 領海 的 水域 同時 舉行 “ 海洋 冒險 ” 等 ３ \n                           個 軍事 演習 。 參 加演 習 的 美軍有 第 ８ ２ 空降 師 ， 第 １ ０ １ \n                           空襲師 和 第 ２ ４ 機械化 步兵 師 ， 數 百架 戰鬥機 和 轟炸機 ， \n                           以及 許多 戰艦 。 \n                                   另據 古巴 官方消息 說 ， 美國 ４ 月 ３ ０ 日 又 向 它 霸 佔 的 \n                           古巴 關 塔那摩 海 軍 基地 增派 了 艦艇 ， 使在關 塔那摩 海軍基 \n                           地 的 美國 軍艦 達 到 １ ２ 艘 。 （ 完 ） \n \n'

In [93]:
def newscut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)


In [95]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [96]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score (pre, y_test)

1.0

In [97]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
      columns=[n+"(預測)" for n in u],
      index=[n+"(真實)" for n in u ])

,軍事(預測),交通(預測),計算機(預測),環境(預測),藝術(預測),醫藥(預測),教育(預測),經濟(預測),體育(預測),政治(預測)
軍事(真實),10,0,0,0,0,0,0,0,0,0
交通(真實),0,10,0,0,0,0,0,0,0,0
計算機(真實),0,0,10,0,0,0,0,0,0,0
環境(真實),0,0,0,10,0,0,0,0,0,0
藝術(真實),0,0,0,0,10,0,0,0,0,0
醫藥(真實),0,0,0,0,0,10,0,0,0,0
教育(真實),0,0,0,0,0,0,10,0,0,0
經濟(真實),0,0,0,0,0,0,0,10,0,0
體育(真實),0,0,0,0,0,0,0,0,10,0
政治(真實),0,0,0,0,0,0,0,0,0,11


In [103]:
test = input("輸入新聞:")
count = vec.transform([poemcut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
  print(name, "的機率:", p)
print("答案:", trans_r[answer])

輸入新聞:邊緣人
軍事 的機率: 0.06364113107177174
交通 的機率: 0.08597370335962616
計算機 的機率: 0.12786796744333698
環境 的機率: 0.13129645603152112
藝術 的機率: 0.024540183821947004
醫藥 的機率: 0.030722922868781484
教育 的機率: 0.06724468321111134
經濟 的機率: 0.23075759795681103
體育 的機率: 0.17228940664071282
政治 的機率: 0.06566594759438103
答案: 經濟
